In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#in the future, use abbreviations for the column variables names

#globals
datasetNum = 1

# load the dataset
testPath = '/Users/shreejakikkisetti/Desktop/CMaps/test_FD00' + str(datasetNum) + '.txt'
testdf = pd.read_csv(testPath, ' ', header=None)
trainPath = '/Users/shreejakikkisetti/Desktop/CMaps/train_FD00' + str(datasetNum) + '.txt'
traindf = pd.read_csv(trainPath, ' ', header=None)
RULPath = '/Users/shreejakikkisetti/Desktop/CMaps/RUL_FD00' + str(datasetNum) + '.txt'
TestUnitToRUL = pd.read_csv(RULPath, header=None)[0].tolist()

#reformat + add headers
testdf = testdf.loc[:,0:25]
traindf = traindf.loc[:,0:25]
colNames = ['unit #', 'time (cycles)', 'operational setting 1', 'operational setting 2', 'operational setting 3']
for i in range(1,22):
    colNames.append('sensor measurement ' + str(i))
testdf.columns = colNames
traindf.columns = colNames
TestUnitToRUL.insert(0,'NaN') #to make indices match unit #s
#TestRULdf.insert(loc=0,column=1,value=list(range(0,TestRULdf.shape[0])))
#TestRULdf.columns = ['unit #', 'Actual RUL']
#print(traindf.shape)    
#print(TestRULdf) 
#print(traindf)

#calculate TrainUnittoTotalLife
TrainUnittoTotalLife = ['NaN']#first value not defined t omake indices match unit #s
for i in range(traindf['unit #'].min(),traindf['unit #'].max()+1):
    TrainUnittoTotalLife.append(traindf[traindf['unit #'] == i]['time (cycles)'].max()) #first part extracts rows that meet the given condition


In [ ]:
#Population plots on training data

from scipy.interpolate import UnivariateSpline

#compute average values for each parameter for each time interval
#not enough values at high values of t --> lots of noise
timeAvgs = [None]*24#24 dimensions that vary by time
counts = [None]*24
for i in range(0,24):
    timeAvgs[i] = [0]*(max(traindf['time (cycles)']))
    counts[i] = [0]*(max(traindf['time (cycles)']))
for i in range(0,24):
    for j in range(0,traindf.shape[0]):
        timeAvgs[i][traindf['time (cycles)'][j]-1] += traindf[colNames[i+2]][j]
        counts[i][traindf['time (cycles)'][j]-1] += 1
for i in range(0,24):
    for j in range(0,max(traindf['time (cycles)'])):
        timeAvgs[i][j] /= counts[i][j]#counts has to be above 0
        
timeAvgsdf = pd.DataFrame(data=timeAvgs).T
timeAvgsdf.columns = colNames[2:]


#plot

for i in range(1,4):
    subPlt = traindf.plot.scatter(title = 'Population Data OS' + str(i) + ' vs. Time for FD001', x='time (cycles)', y='operational setting ' + str(i), s=0.1, alpha=0.5)
    s = UnivariateSpline(list(range(1,max(traindf['time (cycles)']+1))), timeAvgsdf['operational setting ' + str(i)].tolist())
    xs = np.linspace(0, max(traindf['time (cycles)']), 100)
    ys = s(xs)
    subPlt.plot(xs,ys,color='r')
    #print(timeAvgsdf['operational setting ' + str(i)])
for i in range(1,22):
    subPlt = traindf.plot.scatter(title = 'Population Data SM' + str(i) + ' vs. Time for FD001', x='time (cycles)', y='sensor measurement ' + str(i), s=0.1, alpha=0.5)
    #subPlt.plot(list(range(1,max(traindf['time (cycles)'])+1)),timeAvgsdf['sensor measurement ' + str(i)])
    s = UnivariateSpline(list(range(1,max(traindf['time (cycles)']+1))), timeAvgsdf['sensor measurement ' + str(i)].tolist())
    xs = np.linspace(0, max(traindf['time (cycles)']), 100)
    ys = s(xs)
    subPlt.plot(xs,ys,color='r')
    
    
#need to review vectorization for faster processing (and less code)

In [ ]:
#data pre-processing
#not tested for logic errors

import random

#remove constants
constVar = ['operational setting 3', 'sensor measurement 1', 'sensor measurement 5', 'sensor measurement 10',
            'sensor measurement 16', 'sensor measurement 18', 'sensor measurement 19']
#for var in constVar:
#    testdf = testdf.drop([var],axis=1)
#    traindf = traindf.drop([var],axis=1)
    
first = True
#calculate training data
trainData = []
trainRUL = []
for i in range(traindf['unit #'].min(),traindf['unit #'].max()+1): 
    totTime = TrainUnittoTotalLife[i] #total time until system failure
    for j in range(0,5):#5 samples for each training unit
        entry = []
        endTime = random.randint(1,totTime)  #end time for input data on this engine; random inclusive on both sides
        entry.append(endTime)
        entry = entry + traindf[(traindf['unit #'] == i) & (traindf['time (cycles)'] == 1)].values.tolist()[0][2:]
        entry = entry + traindf[(traindf['unit #'] == i) & (traindf['time (cycles)'] == int(endTime/2+1))].values.tolist()[0][2:]
        entry = entry + traindf[(traindf['unit #'] == i) & (traindf['time (cycles)'] == endTime)].values.tolist()[0][2:] 
        trainData.append(entry)
        trainRUL.append(totTime - endTime)
        first = False
testData = []
testRUL = []
for i in range(testdf['unit #'].min(),testdf['unit #'].max()+1): 
    entry = []
    endTime = testdf[testdf['unit #'] == i]['time (cycles)'].max()
    entry.append(endTime)
    entry = entry + testdf[(testdf['unit #'] == i) & (testdf['time (cycles)'] == 1)].values.tolist()[0][2:]
    entry = entry + testdf[(testdf['unit #'] == i) & (testdf['time (cycles)'] == int(endTime/2+1))].values.tolist()[0][2:]
    entry = entry + testdf[(testdf['unit #'] == i) & (testdf['time (cycles)'] == endTime)].values.tolist()[0][2:]
    testData.append(entry)
    testRUL.append(TestUnitToRUL[i])

#print(trainData[0])
#print(trainRUL[0])
#print(testData[0])
#print(testRUL[0])

In [ ]:
#NN

%load_ext tensorboard
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import datetime

#create model
model = keras.Sequential()
n_features = len(trainData[0]) #number of input features
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))#kernel_initializer = layer weight initializer (what type should I use?)
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))#one output
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])#optimizer controls learning rate

#train model
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(trainData, trainRUL, epochs=10, batch_size=32, verbose=2, callbacks=[tensorboard_callback])
%tensorboard --logdir logs/fit

#evaluate model
loss, mse = model.evaluate(testData, testRUL, verbose=0)#what's loss vs. accuracy




#https://www.tensorflow.org/tensorboard/get_started